# bi-lstm+crf，使用pytorch-crf库实现crf，可cuda加速。

数据集说明：

1: B-BANK 代表银行实体的开始

2: I-BANK 代表银行实体的内部

3: B-PRODUCT 代表产品实体的开始

4: I-PRODUCT 代表产品实体的内部

5: O 代表不属于标注的范围

6: B-COMMENTS_N 代表用户评论（名词）

7: I-COMMENTS_N 代表用户评论（名词）实体的内部

8: B-COMMENTS_ADJ 代表用户评论（形容词）

9: I-COMMENTS_ADJ 代表用户评论（形容词）实体的内部

In [1]:
import pandas as pd
import numpy as np
import sys
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
from torchcrf import CRF
from tqdm import tqdm
torch.manual_seed(1)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('{}能用'.format(device))

cuda能用


In [2]:
from sklearn.model_selection import train_test_split
data = pd.read_csv('./train_data_public.csv')
data.drop('Unnamed: 0', axis=1, inplace=True)
train_data, valid_data = train_test_split(data, test_size = 0.2, random_state=42)
train_data.index = list(range(len(train_data)))
valid_data.index = list(range(len(valid_data)))
test_data = pd.read_csv('./test_public.csv')

In [3]:
train_data

,text,BIO_anno,class,bank_topic
0,封卡只是鸡的原因吗？,B-COMMENTS_N I-COMMENTS_N O O O O O O O O,2,交通银行
1,销卡后45天后等征信更新再申请,B-COMMENTS_N I-COMMENTS_N O O O O O O B-PRODUC...,2,建设银行
2,有8w多的，招行的,O O O O O O B-BANK I-BANK O,2,NaN
3,判决金额8.2，表示你需要还款8.2，至于你是一次性还款，还是分期还款，如果判决书里没有具体...,O O B-COMMENTS_N I-COMMENTS_N O O O O O O O O ...,2,工商银行
4,我的-个人资料-联系地址??，刚瞅了眼我自己的信息，好像也乱了，我都怀疑前几天e分期一直秒拒...,O O O O O O O O O O O O O O O O O O O O O O O ...,2,NaN
...,...,...,...,...
7995,座机：0000－00000000我四大行线上申请都是填这个！没,O O O O O O O O O O O O O O O O O O O O O O O ...,2,NaN
7996,私行，钻石，白金，金卡可以让理财经理挑(但是最多也就是3A/4A连号)，普卡也是可以让理财帮...,O O O O O O O O O O O O O O B-PRODUCT I-PRODUC...,2,工商银行
7997,那你运气真不行??邀请短信很少不提的…,O O O O O B-COMMENTS_ADJ I-COMMENTS_ADJ O O O ...,2,NaN
7998,为什么我感觉这个帖子，我看过了，是说工行的么,O O O O O O O O O O O O O O O O O O B-BANK I-B...,2,NaN


In [4]:
# 看看大概的文本长度，以便确定padding长度
len_text = train_data['text'].apply(lambda x: len(list(x)))
len_text.describe()

count    8000.000000
mean       21.005000
std        17.386108
min         1.000000
25%        10.000000
50%        16.000000
75%        25.000000
max       278.000000
Name: text, dtype: float64

In [5]:
x = train_data['text'][0]
list(x)

['封', '卡', '只', '是', '鸡', '的', '原', '因', '吗', '？']

In [6]:
train_data.info()
valid_data.info()
test_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8000 entries, 0 to 7999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   text        8000 non-null   object
 1   BIO_anno    8000 non-null   object
 2   class       8000 non-null   int64 
 3   bank_topic  6124 non-null   object
dtypes: int64(1), object(3)
memory usage: 570.5+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 0 to 1999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   text        2000 non-null   object
 1   BIO_anno    2000 non-null   object
 2   class       2000 non-null   int64 
 3   bank_topic  1512 non-null   object
dtypes: int64(1), object(3)
memory usage: 78.1+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5093 entries, 0 to 5092
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      5093 non-null 

In [7]:
train_data.head(3)

,text,BIO_anno,class,bank_topic
0,封卡只是鸡的原因吗？,B-COMMENTS_N I-COMMENTS_N O O O O O O O O,2,交通银行
1,销卡后45天后等征信更新再申请,B-COMMENTS_N I-COMMENTS_N O O O O O O B-PRODUC...,2,建设银行
2,有8w多的，招行的,O O O O O O B-BANK I-BANK O,2,NaN


In [8]:
# 把text和标注按单个字分隔开，放进列表
train_data['BIO_anno'] = train_data['BIO_anno'].apply(lambda x:x.split(' '))
valid_data['BIO_anno'] = valid_data['BIO_anno'].apply(lambda x:x.split(' '))
# 将text和标注组合存进元组
train_data['training_data'] = train_data.apply(lambda row: [list(row['text']), row['BIO_anno']], axis=1)
valid_data['validating_data'] = valid_data.apply(lambda row: [list(row['text']), row['BIO_anno']], axis=1)
test_data['testing_data'] = test_data.apply(lambda row: list(row['text']), axis=1)



F:\ML_ENVS\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
F:\ML_ENVS\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
F:\ML_ENVS\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

In [9]:
num = train_data['training_data'].apply(lambda x:type(x[0])!=type([]))

In [10]:
num.sum()

0

In [11]:
training_data_txt = train_data['training_data'].to_list()
validating_data_txt = valid_data['validating_data'].to_list()
testing_data_txt = test_data['testing_data'].to_list()
print('训练集大小：',len(training_data_txt))
print('验证集大小：',len(validating_data_txt))
print('测试集大小：',len(testing_data_txt))

训练集大小： 8000
验证集大小： 2000
测试集大小： 5093


In [12]:
# 定义一些工具函数

# 句子转idx
def prepare_sequence(seq, word2idx):
    idxs = [word2idx[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

def argmax(vec):
    # return the argmax as a python int
    # 返回vec的dim为1维度上的最大值索引
    _, idx = torch.max(vec,axis=1)
    return idx.item()

# Compute log sum exp in a numerically stable way for the forward algorithm
# 前向算法是不断累积之前的结果，这样就会有个缺点
# 指数和累积到一定程度后，会超过计算机浮点值的最大值，变成inf，这样取log后也是inf
# 为了避免这种情况，用一个合适的值clip去提指数和的公因子，这样就不会使某项变得过大而无法计算
# SUM = log(exp(s1)+exp(s2)+...+exp(s100))
#     = log{exp(clip)*[exp(s1-clip)+exp(s2-clip)+...+exp(s100-clip)]}
#     = clip + log[exp(s1-clip)+exp(s2-clip)+...+exp(s100-clip)]
# where clip=max
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

In [13]:
class BiLSTM_CRF(nn.Module):
    def __init__(self, configs):
        super(BiLSTM_CRF, self).__init__()
        self.configs = configs
        # if config.embedding_pretrained is not None:
        #     self.embedding = nn.Embedding.from_pretrained(configs.embedding_pretrained,
        #                                                   freeze=False)  # 表示训练过程词嵌入向量会更新
        # else:

        self.embedding = nn.Embedding(configs.vocab_len, configs.embedding_dim,
                                      padding_idx=configs.word2idx['<PAD>'])  # PAD索引填充

        if configs.bidirectional:
            self.num_directions = 2
        else:
            self.num_directions = 1


        self.rnn = nn.LSTM(input_size=configs.embedding_dim,
                           hidden_size=configs.hidden_size,
                           num_layers=configs.num_layers,
                           batch_first=True,
                           bidirectional=configs.bidirectional)

        self.tag2idx = configs.tag2idx

        # 转换参数矩阵 输入i,j是得分从j转换到i
        self.tagset_size = len(self.tag2idx)
        # 将lstm的输出映射到标记空间
        self.hidden2tag = nn.Linear(configs.hidden_size*self.num_directions, self.tagset_size)  # -> (B, num_class+2)  加上了START END
        self.crf = CRF(num_tags=self.tagset_size,batch_first=True)

    def _init_hidden(self, batchs):  # 初始化h_0和c_0 与GRU不同的是多了c_0（细胞状态）
        h_0 = torch.randn(self.configs.num_layers*self.num_directions, batchs,  self.configs.hidden_size)
        c_0 = torch.randn(self.configs.num_layers*self.num_directions, batchs, self.configs.hidden_size)
        return self._make_tensor(h_0), self._make_tensor(c_0)

    def _get_lstm_features(self, x):
        # x.shape: (bs, num_words)
        x = self.embedding(x)
        # x.shape: (bs, num_words, embedding_dim)
        h_0, c_0 = self._init_hidden(batchs=x.size(0))
        out, (hidden, c) = self.rnn(x,(h_0, c_0))
        # out.shape: (bs, num_words, hidden_size*2)
        out = self.hidden2tag(out)  # (B,num_directions*hidden_size) -> (B, num_class)
        # out.shape: (bs, num_words, tagset_size)
        return out

    def neg_log_likelihood(self, sentence_tensor=None, label_tensor=None, mask_tensor=None):  # 损失函数
        feats = self._get_lstm_features(sentence_tensor)
        return -self.crf(emissions=feats, tags=label_tensor, mask=mask_tensor)
        # return -self.crf(emissions=feats, tags=label_tensor)


    def _make_tensor(self, tensor):
        # 函数说明： 将传入的tensor转移到cpu或gpu内
        tensor_ret = tensor.to(self.configs.device)
        return tensor_ret


    def forward(self, sentence_tensor=None, mask_tensor=None):
        # 数据预处理时，x被处理成是一个tuple,其内容是: (word, label).
        # x:b_size
        # print(sentence_tensor)
        # print(mask_tensor)
        lstm_feats = self._get_lstm_features(sentence_tensor)  # 获取BiLSTM的emission分数

        # Returns: List of list containing the best tag sequence for each batch.
        # 返回列表组成的标签

        out = self.crf.decode(emissions=lstm_feats,
                              mask=mask_tensor)
        return out

In [14]:
from utils.param_configs import Configs
configs = Configs()

# 将训练集汉字使用数字表示
# 为了方便调试，先用100条数据进行训练，调试好后可用全量数据进行训练
# training_data_txt = training_data_txt[:]
# --------------------------建立字典，字: idx-------------------------------------
word2idx = {}
# 训练集的
for sentence, tags in training_data_txt:
    for word in sentence:
        if word not in word2idx:
            word2idx[word] = len(word2idx)

# 验证集的
for sentence, tags in validating_data_txt:
    for word in sentence:
        if word not in word2idx:
            word2idx[word] = len(word2idx)

# 测试集的
testing_data = testing_data_txt
for sentence in testing_data:
    for word in sentence:
        if word not in word2idx:
            word2idx[word] = len(word2idx)

# 加2个特殊字符
word2idx['<UNK>'] = len(word2idx)
word2idx['<PAD>'] = len(word2idx)

configs.word2idx = word2idx
configs.vocab_len = len(word2idx)
# ------------------------------------------------------------------------------

In [15]:
len(training_data_txt[0][0]),len(training_data_txt[0][1])
len(training_data_txt)

8000

In [16]:
from utils.data_process import create_data_loader
train_data_loader = create_data_loader(training_data_txt, configs)
valid_data_loader = create_data_loader(validating_data_txt, configs)
# test_data_loader = create_data_loader(testing_data_txt, configs) # 没有标签的测试集就不这样构建，因为没有label

In [17]:
len(train_data_loader),len(valid_data_loader)

(500, 125)

In [18]:
for sample in train_data_loader:
    print(sample)
    break


{'sentence': ['封 卡 只 是 鸡 的 原 因 吗 ？ <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>', '销 卡 后 4 5 天 后 等 征 信 更 新 再 申 请 <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>', '有 8 w 多 的 ， 招 行 的 <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>', '判 决 金 额 8 . 2 ， 表 示 你 需 要 还 款 8 . 2 ， 至 于 你 是 一 次 性 还 款 ， 还', '我 的 - 个 人 资 料 - 联 系 地 址 ? ? ， 刚 瞅 了 眼 我 自 己 的 信 息 ， 好 像 也 乱', '这 短 信 只 是 系 统 统 一 默 认 的 ， 短 信 提 示 制 卡 中 其 实 也 就 是 等 于 数 字 卡', '我 固 定 额 度 3 0 0 0 0 , 现 金 分 期 1 2 0 0 0 0 ！ <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>', '求 哥 哥 帮 忙 。 到 了 这 个 界 面 还 会 被 拒 绝 吗 ？ 刷 脸 要 等 2 4 小 时 。 <PAD> <PAD>', '最 近 我 刷 了 好 几 次 都 没 有 ， 机 器 换 了 几 台 也 没 有 封 卡 啊 <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>', '以 前 经 常 空 卡 ， 今 年 开 始 正 常 用 卡 ， 真 实 加 套 现 ， 留 2 0 % 以 上 额 度', '招 商 哪 个 大 功 能 需 要 盾 <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

In [19]:
len(train_data_loader)

500

In [20]:

configs.word2idx['封']

0

In [21]:
sample['sentence']

['封 卡 只 是 鸡 的 原 因 吗 ？ <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>',
 '销 卡 后 4 5 天 后 等 征 信 更 新 再 申 请 <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>',
 '有 8 w 多 的 ， 招 行 的 <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>',
 '判 决 金 额 8 . 2 ， 表 示 你 需 要 还 款 8 . 2 ， 至 于 你 是 一 次 性 还 款 ， 还',
 '我 的 - 个 人 资 料 - 联 系 地 址 ? ? ， 刚 瞅 了 眼 我 自 己 的 信 息 ， 好 像 也 乱',
 '这 短 信 只 是 系 统 统 一 默 认 的 ， 短 信 提 示 制 卡 中 其 实 也 就 是 等 于 数 字 卡',
 '我 固 定 额 度 3 0 0 0 0 , 现 金 分 期 1 2 0 0 0 0 ！ <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>',
 '求 哥 哥 帮 忙 。 到 了 这 个 界 面 还 会 被 拒 绝 吗 ？ 刷 脸 要 等 2 4 小 时 。 <PAD> <PAD>',
 '最 近 我 刷 了 好 几 次 都 没 有 ， 机 器 换 了 几 台 也 没 有 封 卡 啊 <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>',
 '以 前 经 常 空 卡 ， 今 年 开 始 正 常 用 卡 ， 真 实 加 套 现 ， 留 2 0 % 以 上 额 度',
 '招 商 哪 个 大 功 能 需 要 盾 <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <P

In [22]:
sample['label']


['B-COMMENTS_N I-COMMENTS_N O O O O O O O O O O O O O O O O O O O O O O O O O O O O',
 'B-COMMENTS_N I-COMMENTS_N O O O O O O B-PRODUCT I-PRODUCT O O O O O O O O O O O O O O O O O O O O',
 'O O O O O O B-BANK I-BANK O O O O O O O O O O O O O O O O O O O O O O',
 'O O B-COMMENTS_N I-COMMENTS_N O O O O O O O O O B-COMMENTS_N I-COMMENTS_N O O O O O O O O O O O B-COMMENTS_N I-COMMENTS_N O O',
 'O O O O O O O O O O O O O O O O O O O O O O O O O O O O O B-COMMENTS_ADJ',
 'O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O',
 'O O O B-COMMENTS_N I-COMMENTS_N O O O O O O O O B-PRODUCT I-PRODUCT O O O O O O O O O O O O O O O',
 'O O O O O O O O O O O O O O O B-COMMENTS_ADJ I-COMMENTS_ADJ O O O O O O O O O O O O O',
 'O O O O O O O O O O O O O O O O O O O O O B-COMMENTS_N I-COMMENTS_N O O O O O O O',
 'O O O O O O O O O O O B-COMMENTS_ADJ I-COMMENTS_ADJ O O O O O O O O O O O O O O O B-COMMENTS_N I-COMMENTS_N',
 'B-BANK I-BANK O O O O O O O O O O O O O O O O O O O O O O O O O O O O',
 'O

In [23]:
sample['sentence_tensor']

tensor([[   0,    1,    2,    3,    4,    5,    6,    7,    8,    9, 2621, 2621,
         2621, 2621, 2621, 2621, 2621, 2621, 2621, 2621, 2621, 2621, 2621, 2621,
         2621, 2621, 2621, 2621, 2621, 2621],
        [  10,    1,   11,   12,   13,   14,   11,   15,   16,   17,   18,   19,
           20,   21,   22, 2621, 2621, 2621, 2621, 2621, 2621, 2621, 2621, 2621,
         2621, 2621, 2621, 2621, 2621, 2621],
        [  23,   24,   25,   26,    5,   27,   28,   29,    5, 2621, 2621, 2621,
         2621, 2621, 2621, 2621, 2621, 2621, 2621, 2621, 2621, 2621, 2621, 2621,
         2621, 2621, 2621, 2621, 2621, 2621],
        [  30,   31,   32,   33,   24,   34,   35,   27,   36,   37,   38,   39,
           40,   41,   42,   24,   34,   35,   27,   43,   44,   38,    3,   45,
           46,   47,   41,   42,   27,   41],
        [  67,    5,   68,   69,   70,   71,   72,   68,   73,   74,   75,   76,
           77,   77,   27,   78,   79,   80,   81,   67,   82,   83,    5,   17,
      

In [24]:
sample['mask_tensor'][0]

tensor([ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False])

In [25]:
sample['label_tensor'][-6]

tensor([1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0])

# <font color='red'>Model Training

In [26]:
def train_epoch(model, data_loader, optimizer, scheduler, configs):
    # 训练模式
    model = model.train()
    loss_list = []
    for sample in tqdm(data_loader, 'train'):
        sentence_tensor = sample['sentence_tensor'].to(configs.device)
        mask_tensor = sample['mask_tensor'].to(configs.device)
        label_tensor = sample['label_tensor'].to(configs.device)
        # print(sentence_tensor)
        # print(mask_tensor)
        # print(label_tensor)
        out = model(sentence_tensor, mask_tensor)
        # print(out)
        loss = model.neg_log_likelihood(sentence_tensor=sentence_tensor,
                                        label_tensor=label_tensor,
                                        mask_tensor=mask_tensor)
        loss_list.append(loss.item())
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    return np.mean(loss_list)


def eval_epoch(model, data_loader, configs):
    # 验证模式
    model = model.eval()
    loss_list = []
    # 关闭自动求导，省内存加速，因为是不是训练模式了，没必要求导
    with torch.no_grad():
        for sample in tqdm(data_loader, 'val'):
            sentence_tensor = sample['sentence_tensor'].to(configs.device)
            mask_tensor = sample['mask_tensor'].to(configs.device)
            label_tensor = sample['label_tensor'].to(configs.device)
            out = model(sentence_tensor, mask_tensor)
            loss = model.neg_log_likelihood(sentence_tensor=sentence_tensor,
                                            label_tensor=label_tensor,
                                            mask_tensor=mask_tensor)
            loss_list.append(loss.item())


    return np.mean(loss_list)

In [27]:
model = BiLSTM_CRF(configs).to(device)
# 优化器
optimizer = optim.Adam(model.parameters(), lr=0.001)
# 学习率指数衰减  每个epoch: lr = gamma*lr
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
# 损失函数。这个倒是不用定义了，因为pytorch-crf的crf自带loss的计算
# loss_fn = nn.CrossEntropyLoss()


EPOCHS = 2

In [28]:
for epoch in range(EPOCHS):
    print('——'*10, f'Epoch {epoch + 1}/{EPOCHS}', '——'*10)
    train_loss = train_epoch(model, train_data_loader, optimizer,scheduler, configs)
    print(f'Train loss : {round(train_loss, 2)}')

    val_loss = eval_epoch(model, valid_data_loader, configs)
    print(f'Val loss : {round(val_loss, 2)}')

———————————————————— Epoch 1/2 ————————————————————
Train loss : 236.79
Val loss : 235.08
———————————————————— Epoch 2/2 ————————————————————
Train loss : 229.41
Val loss : 234.94


val: 100%|██████████| 125/125 [00:02<00:00, 48.41it/s]


In [29]:
# predict_tag = []
# for l in out:
#     temp = []
#     for i in l:
#         temp.append(configs.idx2tag[i])
#     predict_tag.append(temp)


# <font color='red'>test some instances

In [29]:
# 验证模式
model = model.eval()
loss_list = []
# 关闭自动求导，省内存加速，因为是不是训练模式了，没必要求导
with torch.no_grad():
    for sample in tqdm(valid_data_loader, 'val'):
        sentence_tensor = sample['sentence_tensor'].to(configs.device)
        mask_tensor = sample['mask_tensor'].to(configs.device)
        label_tensor = sample['label_tensor'].to(configs.device)
        out = model(sentence_tensor, mask_tensor)
        predict_tag = []
        for l in out:
            temp = []
            for i in l:
                temp.append(configs.idx2tag[i])
            predict_tag.append(temp)
        break

val:   0%|          | 0/125 [00:00<?, ?it/s]


In [31]:

len(predict_tag[0]),len(predict_tag[1]),len(predict_tag[2])

(15, 13, 30)

In [30]:
out


[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [47]:
sample['sentence_tensor']


tensor([[  63,   29,  191,  148, 1826],
        [ 178,   53,   60,   61,  283],
        [1118,   14,  532,   17,    5],
        [ 127,  222,   40,  140,   69],
        [ 473,   38,   69,  100,  952],
        [ 479,  532,   17,   27,  372],
        [1050,   54,   23,  215,  367],
        [1466,  527,    2,  181,  167],
        [ 115,   29,  547,  175,   23],
        [ 718,   92,   41,  191,  145],
        [ 450,   27,   64,   65,   85],
        [ 183,  184,    3,  382,   14],
        [  33,  135,   60,   61,   27],
        [ 709,   45,  656,  709,  386],
        [ 508,  190,   33,  135,   41],
        [  38,    1,  145,   80,    8]])

In [33]:
sample['mask_tensor']


tensor([[ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         False, False, False, False, False, False, False, False, False, False],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True, False, False, False, False, False],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True, False,
         False, False, False, False, False, False, False, False, False, False],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  T

In [32]:
sample['label_tensor']

tensor([[0, 0, 0, 0, 7, 8, 8, 8, 0, 0, 0, 7, 8, 8, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4,
         7, 8, 8, 0, 0, 0],
        [7, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 0,
         0, 0, 0, 7, 0, 0],
        [0, 0, 0, 0, 3, 4, 4, 0, 3, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0],
        [7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0],
        [0, 0, 7, 8, 0, 3, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0],
        [1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0

In [33]:
sample['sentence']


['银 行 不 会 雪 中 送 炭 ， 只 会 锦 上 添 花 <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>',
 '哪 里 可 以 看 到 我 有 没 有 拒 绝 ？ <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>',
 '昨 天 微 信 的 上 海 信 访 这 个 月 七 号 上 门 了 协 商 个 性 化 分 期 不 同 意 说 是 他',
 '最 少 要 1 个 半 月 ， 我 以 前 处 理 过 。 <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>',
 '教 你 个 办 法 ， 信 不 信 由 你 ， 先 去 柜 台 人 工 申 请 e 分 期 ， 通 过 率 高 ， 只',
 '绑 微 信 ， 支 付 宝 ， 云 闪 付 <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>',
 '冷 没 有 过 去 ? ? <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>',
 '境 外 只 能 用 支 付 宝 微 信 <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>',
 '中 行 每 月 有 一 次 5 元 立 减 金 <PAD> <PAD> <PAD> <PAD> <P